**Data Gathering and Preprocessing**


In [ ]:
# download and read in json files
import json
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer

nlp = spacy.load("en_core_web_sm")

def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

file1_path = 'squad_dev_set.json'
file2_path = 'squad_train_set.json'
data1 = load_json(file1_path)
data2 = load_json(file2_path)

combined_data = data1['data'] + data2['data']

In [11]:
#Convert Questions and Answers to Dataframe
preprocessed_data = []
i = 0
for entry in combined_data:
    for paragraph in entry['paragraphs']:
        for qa in paragraph['qas']:
            preprocessed_data.append({
                'topic_id': i,
                'cleaned_question': qa['question'],
                'answer': qa['answers'][0]['text'] if qa['answers'] else None
            })
    i+=1

preprocessed_data_final = [qanda for qanda in preprocessed_data if qanda['answer'] is not None]
pd.DataFrame(preprocessed_data_final).to_csv("qAndA.csv")

In [ ]:
#!pip install wikipedia

In [13]:
import wikipedia
import pandas as pd

In [ ]:
#Identify Relevant wikipedia pages
topics = list(pd.DataFrame(combined_data)['title'])
wikiTopics = []
i = 0
#(rakshitarora)
for topic in topics:
    searchTopic = re.sub(r'%A9', 'e', topic)
    searchTopic = re.sub(r'%..', '', searchTopic).strip()
    print(searchTopic, i)
    if len(searchTopic) >= 1:
        topico = wikipedia.search(searchTopic)[0]
        wikiTopics.append(topico)
    i+=1

In [ ]:
#Pull Wikipedia page data
#Chat Assistance
#(rakshitarora)
pages = []
i = 0
for topic in wikiTopics:
    try:
        print(topic)
        pages.append(wikipedia.page(topic))
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation error for topic: {topic}. Options: {e.options}")
        pages.append(None)
    except wikipedia.exceptions.PageError as e:
        print("Page not found")
        pages.append(None)
    i+=1

In [ ]:
#Extract information from HTML
#Chat
from bs4 import BeautifulSoup

# Assume `html_content` is the HTML of a Wikipedia page
def get_main_body_text(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Locate the main content div
    main_content = soup.find('div', class_='mw-parser-output')
    
    # Extract the text from paragraphs within the main content
    if main_content:
        paragraphs = main_content.find_all('p')
        body_text = ' '.join([p.get_text(strip=False) for p in paragraphs])
        return body_text
    else:
        return None


context = []
i = 0
for page in pages:
    if page != None:
        text = get_main_body_text(page.html())
        if text != None:
            context.append(text)
    else:
        context.append(None)
    i += 1
    print(i)
    
context


In [ ]:
#Clean context

contexto = context.copy()
def clean_text(content):
    content = re.sub(r'<.*?>', '', content)
    content = re.sub(r'\[\d+\]', '', content)
    #content = re.sub(r'.', '', content)
    content = re.sub(r'\s+', ' ', content)
    return content

for i in range(len(contexto)):
    if contexto[i] != None:
        contexto[i] = clean_text(contexto[i])
    else:
        contexto[i] = ''
#ChatGPT
with open('context.txt', "w") as file:
    for line in contexto:
        file.write(line + "\n")

print(contexto)


In [28]:
#Randomly select questions with available context
#(Janiszewski)
from random import random
question_answer = []
indices = len(preprocessed_data_final)
while len(question_answer) < 25:
    index = int(random() * indices)
    qa = preprocessed_data_final[index]

    if context[qa['topic_id']] != None:
        question_answer.append(qa)

pd.DataFrame(question_answer).to_csv("questions.csv", index=False)